In [7]:
import time
import qpsolvers as qp
import deepSI
import numpy as np
from casadi import *
from torch import nn
from matplotlib import pyplot as plt

import Systems
from my_rk4 import *
from lpv_int import *
from par_lpv_int import *
from lpv_rk4 import *
from mpcUtil import *
import torch

In [8]:
dt = 0.1
Nc = 25
Nsim = 30
#dlam = 0.05
stages = 2
dlam = 1.0/stages
max_iterations = 4

# Weight matrices for the cost function
Q = np.matrix('10,0;0,10')
R = 1

In [9]:
sys_unblanced = Systems.LTI()
encoder = deepSI.load_system("systems/LtiTanh_nab_4_e100")

In [12]:
nx = encoder.nx
n_states = nx
x = MX.sym("x",nx,1)
nu = encoder.nu if encoder.nu is not None else 1
n_controls = nu
u = MX.sym("u",nu,1)
ny = encoder.ny if encoder.ny is not None else 1

# convert torch nn to casadi function
rhs = CasADiFn(encoder, x, u)
f = Function('f', [x, u], [rhs])
y_rhs = CasADiHn(encoder, x)
h = Function('h', [x], [y_rhs])

correction = f([0,0,0,0], 0)
rhs_c = rhs - correction
correction_h = h([0,0,0,0])
y_rhs_c = y_rhs - correction_h

In [16]:
np.zeros((nx,))

array([0., 0., 0., 0.])